In [ ]:
################################################
# New style 
################################################
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.Plotting import utils as puti
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( puti )
importlib.reload( nuti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)


In [ ]:
from myPythonTools.YAML import get_exp_list as Xl
importlib.reload(Xl)


In [ ]:
print( sys.path )
############################
# Initialize dict for regridding
############################
RgObs={}


In [ ]:
### Make a list of files
import yaml
#from box import Box


with open('../YAML/configure_plotting.yaml', 'r') as file:
    cfgPlot = yaml.safe_load(file)


######################################################
season =  'jja' #'jja' #cfgPlot['zonal_mean']['season']
valData  = cfgPlot['zonal_mean']['validation']['data']
valYears = cfgPlot['zonal_mean']['validation']['years']
print( f"Season= {season}" )
print( valData , valYears )
fld='T' #'U'


regrid_val_x_model = False # True
all_x_fv1x1_zERA = False #True #True
use_desc_in_title = True
use_pmid_for_z =False
plot_jet_axis_range =True

In [ ]:

#Xl.Fig1Paper()  #JointGWTune()  #Utility() 

exp_file,xl= Xl.Ctl_c6477() #Xl.Fig1Paper()  #JointGWTune()  #Utility()
exp_file,xl= Xl.KeyFig_Paper() 
#exp_file,xl= Xl.RadTest() 
#exp_file,xl= Xl.JointGWTune()  

exp_file=f'../YAML/{exp_file}'
with open( exp_file, 'r') as file:
    cfg = yaml.safe_load(file)

Al =[]
for x in xl:
    exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
    ymdPat = '*' #cfg[x]['ymdPat']
    print( exp, subd, Src, Hkey, Dst, useri )
    print( ymdPat  )
    
    A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                        hsPat='cam.h0a' , ymdPat=ymdPat, cmip_fld=fld,
                        verbose=True, open_dataset=True )
    A['alias']=x
    A['Src']=Src
    if ('SrcVgrid' in cfg[x]):
        A['SrcVgrid']=cfg[x]['SrcVgrid']
        print( f"SrcVgrid for {x} = {A['SrcVgrid']} ")
    else:
        print( f'No SrcVgrid in {x}' )
    A['Hkey']=Hkey
    if 'gw_parameters' in cfg[x]:
        A['gw_parameters'] = cfg[x]['gw_parameters']
    if 'desc' in cfg[x]:
        A['desc'] = cfg[x]['desc']
    Al.append( A )
    print( f'Vertcal levs - {A.X.lev.shape}' )
    print( f'{A.exp} Years {len(A.X.time) /12. } \n')


In [ ]:
########################################
# Make seasonal means
########################################

for A in Al:
    print( f'{A.exp} Years {len(A.X.time) /12. }')
    
    AAa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld=fld,return_time=True)
    A['AA_s']=AAa_s
    A['years']=years_a
    A['months']=months_a
    PSa_s = Av.Seasonal( ds=A.X, season=season , fld='PS' )
    A['PS_s']=PSa_s

    if (use_pmid_for_z==True):
        PMIDa_s = Av.Seasonal( ds=A.X, season=season , fld='PMID' )
        A['PMID_s']=PMIDa_s



In [ ]:

########################################
# Make zonal means
########################################
method='C2'

for A in Al:
    Hkey = A['Hkey']
    Src  = A['Src']
    AAa_s = A['AA_s']
    PSa_s = A['PS_s']
    if (use_pmid_for_z==True):
        PMIDa_s = A['PMID_s']
    
    if (Hkey == 'yx' ):
        ########################
        # Option 1 from lat-lon regirrdded data
        ########################

        lon_a=A.X.lon.values
        lat_a=A.X.lat.values
        lev_a=A.X.lev.values
        zlev_a=-7.0*np.log( lev_a/1_000. )

        AAa_sz=np.average( AAa_s , axis=2 )
        if (use_pmid_for_z==True):
            PMIDa_sz = np.average( PMIDa_s , axis=2 )
            zlev_a=-7.0*np.log( PMIDa_sz / 100_000. )
    
    elif (Hkey=='c'):
        ########################
        # Option 2 from unstructured SE data
        ########################
        
        Dst = 'fv1x1'
        DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
        lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
        lev_a=A.X.lev.values
        zlev_a=-7.0*np.log( lev_a/1_000. )
    
        RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=Src, dst=Dst)
        AAa_s_xy=RgF.Horz(xfld_Src=AAa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )

        # Zonal Av
        AAa_sz=np.average( AAa_s_xy , axis=2 )

        if (use_pmid_for_z==True):
            print( Src , Dst )
            print( 'PMIDa_s', np.shape(PMIDa_s) )
            PMIDa_s_xy=RgF.Horz(xfld_Src=PMIDa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
            print( 'PMIDa_s_xy', np.shape(PMIDa_s_xy) )
            PMIDa_sz = np.average( PMIDa_s_xy , axis=2 )
            zlev_a=-7.0*np.log( PMIDa_sz / 100_000. )
            print( 'PMIDa_sz',np.shape(PMIDa_sz) )
            print( 'zlev_a',np.shape(zlev_a) )

    A['AA_sz'] = AAa_sz
    A['lonx'] = lon_a
    A['latx'] = lat_a
    A['zlev'] = zlev_a
    A['plev'] = lev_a
    A['DateRange'] = f"{(A.X.time[0].values.item() ).strftime('%Y-%m-%d')}_{(A.X.time[-1].values.item() ).strftime('%Y-%m-%d')}"

In [ ]:
####
# Validation
####

valYears='*'
Aval = Val.data(fld=fld,season=season,zlev=True , Years=valYears ) #, Years='1996' ) #, Years='2000' )

print( " shape of Val data ",Aval['aa'].shape )
if (Aval['data_exists']==True):
    print( f'Found validation data for {fld}' )
    AAv_s , zlev_v,lat_v,lon_v, years_Uv, Src, SrcVgrid = Aval['aa'], Aval['lev'], Aval['lat'], Aval['lon'], Aval['years'], Aval['data_Hgrid'], Aval['data_Vgrid']
    
    AAv_sz=np.average( AAv_s , axis=2 )
    A['AAv_sz']=AAv_sz
        
    Aval['AA_sz']=AAv_sz
    Aval['lon'] = lon_v
    Aval['lat'] = lat_v
    Aval['zlev'] = zlev_v
    
else:
    print( f'No validation data for {fld}' )



In [ ]:
#print(np.shape(Al[1]['PMID_s']))
from myPythonTools.Zany import jetaxis as jax
importlib.reload( jax )


if ( fld=='U' and season=='jja' and plot_jet_axis_range==True):
    
    for A in Al:
        jaxes,mean,min,max,std,nyears = jax.make_axes_mean_range( X=A.X, Src=A.Src )
        
        A['jaxes']=jaxes
        A['jax_mean']=mean
        A['jax_min']=min
        A['jax_max']=max
        A['jax_std']=std
        A['jax_nyears']=nyears

        print( f'Years in {A.exp} = {nyears}' )
    
    Xval =xr.open_mfdataset( '/glade/campaign/cgd/amp/juliob/ERA5/fv1x1_monthly/ZY/ERA5.fv1x1_zERA_zonal.time*.nc' )
    
    uval=Xval.U.values
    print( uval.shape )
    
    nt = np.shape( uval )[0]
    nyears_val = nt // 12
    
    print( nyears_val )
        
    iseason = np.array( [6,7,8] ) - 1
    
    uval_s = np.stack([
        np.average(uval[y*12 + iseason , :, :], axis=0)
        for y in np.arange(nyears_val)
    ])
    
    
    
    jaxes_val=jax.lat_z( u=uval_s, lat=lat_a )
    
    mean_val = np.mean(jaxes_val, axis=0)
    min_val = np.min(jaxes_val, axis=0)
    max_val = np.max(jaxes_val, axis=0)
    std_val = np.std(jaxes_val, axis=0)
    
    Aval['jaxes']=jaxes_val
    Aval['jax_mean']=mean_val
    Aval['jax_min']=min_val
    Aval['jax_max']=max_val
    Aval['jax_std']=std_val
    Aval['jax_nyears']=nyears_val


In [ ]:
# Get information for Common grid ... here = fv1x1 x L93

CommonGridInfo = GrU.gridInfo(grid='fv1x1', Vgrid='L93' )
lat_comm,lon_comm   = GrU.latlon( scrip= CommonGridInfo['scrip'], Hkey=CommonGridInfo['Hkey'] )
plev_comm,zlev_comm = GrU.vgrid( VgridFile= CommonGridInfo['VgridFile'] )
Lat_comm , Zlev_comm = np.meshgrid(lat_comm, zlev_comm)
CommonGrid ={ 'lat': Lat_comm , 'zlev':Zlev_comm }


In [ ]:
Lat_v , Zlev_v = np.meshgrid(lat_v, zlev_v)
poop = nuti.ZsY_x_ZtY( y_src=Zlev_v , x_src=Lat_v , data_src= AAv_sz , y_tgt=CommonGrid['zlev'], x_tgt=CommonGrid['lat'] )

print(poop.shape)


plt.contour( Lat_comm, Zlev_comm , poop - Al[1].AA_sz )

print( zlev_comm )

In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))

numx=len( Al )
plot_bias_vs_obs = False #True
if (plot_bias_vs_obs == True):
    nxplo,nyplo=numx+1,1
else:
    nxplo,nyplo=numx+1,1

latlims = (-90.,0. )

fig = plt.figure(figsize=(8*nxplo, 9))
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

ulev=np.linspace( -60,140,num=21) 
ulevx = np.append(ulev, [160,180,200])

ulev=np.linspace( -0.05,0.05,num=41) 
ulevx=ulev


####################################
# Use a centarlized functiin to define 
# intervals 
####################################
#def contour_intervals( fld, **kwargs ): return flev,dlev,scale

#dlev=np.linspace( -20,20,num=21) 

ulev,dlev,scale = puti.contour_intervals( fld, zonal_mean=1 )
#ulev=np.linspace(-450,450,num=51)
#ulev=np.linspace(-5,5,num=51)
ulevx=ulev
print(dlev)
cmap='bwr_r' #'gist_ncar'
cmap='gist_ncar'
titlesize=14 #24 #18

plotTop=85 #40 # 15 #10 #85
BoxParm =['alpha_gw_movmtn', 'effgw_rdg_resid','use_gw_rdg_resid']
BoxParm =['use_gw_convect_dp','use_gw_movmtn_pbl','use_gw_front','use_gw_rdg_resid','use_gw_rdg_beta','use_gw_oro']
BoxParm =['use_gw_movmtn_pbl','use_gw_rdg_resid','movmtn_source','alpha_gw_movmtn', 'effgw_movmtn_pbl','effgw_rdg_resid',]
BoxParm =['movmtn_source','alpha_gw_movmtn', 'effgw_movmtn_pbl','taubgnd','effgw_cm','effgw_rdg_beta','effgw_rdg_resid',]
#BoxParm =['use_gw_movmtn_pbl','movmtn_source','alpha_gw_movmtn', 'effgw_movmtn_pbl',]  #'effgw_rdg_resid',]


titles=[ r'1$^\circ$ control','14km simulation',r'1$^\circ$ w/ additional GW source','yy',]
titles=[ r'1$^\circ$ control',r'1$^\circ$ w/out GW parameterization','yy',]
ix=0

for A in Al:
    
    AAa_sz = A['AA_sz'] 
    if (Aval['data_exists']==True) and ((regrid_val_x_model == True) or (all_x_fv1x1_zERA==True)) and (plot_bias_vs_obs==True) :
        AAa_sz = AAa_sz - A['AAv_sz']
        mlev=dlev
        cmap='bwr' #_r'
        plotTop=65 #40 # 15 #10 #85
    else:
        mlev=ulev
        cmap='gist_ncar'
        plotTop=85 #85 #40 # 15 #10 #85
        
    
    lon_a = A['lonx']
    lat_a = A['latx']
    zlev_a = A['zlev']
    years_a = A['years']
    if 'gw_parameters' in A:
        parameters=A['gw_parameters']
    else:
        parameters=None

    if (use_pmid_for_z==True):
        Zlev_a = zlev_a
        # Create 2D arrays
        lat_a, Lev_a = np.meshgrid(lat_a, A.X.lev.values )
    else:
        # Create 2D arrays
        Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
    
    # Adding the hatching for the sponge layer
    sponge_layer = np.zeros_like(AAa_sz)
    sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1

    if ('desc' in A) and (use_desc_in_title == True):
        title = f"{A.desc} \n {season.upper()} {years_a[0]}-{years_a[-1]}"
    else:
        title = f"<{A.exp}> \n {season.upper()} {years_a[0]}-{years_a[-1]}"
    #title = titles[ix]

    n=ix+1
    Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=0.8 ) 
    ax1 = fig.add_axes( Axes1 )
    co1=ax1.contourf(lat_a,zlev_a,AAa_sz ,levels=mlev, cmap=cmap ,extend='both' )
    co2=ax1.contour(lat_a,zlev_a,AAa_sz ,levels=mlev, colors='black')
    #ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
    ax1.set_title( title ,fontsize=titlesize )
    ax1.set_ylim(0,plotTop)
    ax1.set_xlabel('Latitude', fontsize=16 )
    ax1.set_ylabel('Z (km)', fontsize=16 )
    cb=plt.colorbar(co1)
    ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
        fontsize=20, fontweight='bold', va='top')
    
    # Sponge layer hatching
    ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

    ax1.set_xlim( latlims )

    if ( 'jaxes' in A):
        #ax1.plot( A.jax_mean , zlev_a , linewidth=2.5, color='black')
        ax1.plot( A.jax_min , zlev_a , linewidth=2.5, color='black' , linestyle='--')
        ax1.plot( A.jax_max , zlev_a , linewidth=2.5, color='black' , linestyle='--')

    
    ix=ix+1

if (Aval['data_exists']==True):
    for ix in np.arange( start=numx,stop=numx+1 ):

        if (regrid_val_x_model == True) or (all_x_fv1x1_zERA==True):
            lon_v = Al[0]['lon_v']
            lat_v = Al[0]['lat_v']
            zlev_v = Al[0]['zlev_v']
            AAv_sz = Al[0]['AAv_sz']
            
        cmap='gist_ncar'
        title = f"<{valData}> {season.upper()} {years_Uv} " 
        n=ix+1
        Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=0.8 ) 
        ax1 = fig.add_axes( Axes1 )
        co1=ax1.contourf(lat_v,zlev_v,AAv_sz ,levels=ulev, cmap=cmap , extend='both' )
        co2=ax1.contour(lat_v,zlev_v,AAv_sz ,levels=ulev, colors='black')
        ax1.set_title( title ,fontsize=titlesize )
        ax1.set_ylim(0,plotTop)
        ax1.set_xlabel('Latitude', fontsize=16 )
        ax1.set_ylabel('Z (km)', fontsize=16 )
        cb=plt.colorbar(co1)
        ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
            fontsize=20, fontweight='bold', va='top')
        ax1.set_xlim( latlims )
        
        if ( 'jaxes' in Aval):
            #ax1.plot( Aval['jax_mean'] , zlev_v , linewidth=2.5, color='black')
            ax1.plot( Aval['jax_min'] , zlev_v , linewidth=2.5, color='black' , linestyle='--')
            ax1.plot( Aval['jax_max'] , zlev_v , linewidth=2.5, color='black' , linestyle='--')


FigName = f"AGU_2024_Cecile_poster_zonalUT.png" #f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
FigName = f"multiFig.png" #f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
FigName = f"Figure_1.png" #f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
FigName = f"Fig_KeyResult4paper_{fld}.png"

print( FigName) 
Pu.save_link(png_filename = FigName, dpi=600 )

n_next_startlabel = n


In [ ]:
Lat_v , Zlev_v = np.meshgrid( Aval['lat'], Aval['zlev'] )
AAv_sz_comm = nuti.ZsY_x_ZtY( y_src=Zlev_v , x_src=Lat_v , data_src= AAv_sz , y_tgt=CommonGrid['zlev'], x_tgt=CommonGrid['lat'] )

Aval['AA_sz_comm'] = AAv_sz_comm 

In [ ]:
#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))

numx=len( Al )
plot_bias_vs_obs = False #True
if (plot_bias_vs_obs == True):
    nxplo,nyplo=numx+1,1
else:
    nxplo,nyplo=numx+1,1

latlims = (-90.,0. )

fig = plt.figure(figsize=(8*nxplo, 9))
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

ulev=np.linspace( -60,140,num=21) 
ulevx = np.append(ulev, [160,180,200])

ulev=np.linspace( -0.05,0.05,num=41) 
ulevx=ulev


####################################
# Use a centarlized functiin to define 
# intervals 
####################################
#def contour_intervals( fld, **kwargs ): return flev,dlev,scale

#dlev=np.linspace( -20,20,num=21) 

ulev,dlev,scale = puti.contour_intervals( fld, zonal_mean=1 )
#ulev=np.linspace(-450,450,num=51)
#ulev=np.linspace(-5,5,num=51)
ulevx=ulev
print(dlev)
cmap='bwr_r' #'gist_ncar'
cmap='gist_ncar'
titlesize=14 #24 #18

plotTop=65 #40 # 15 #10 #85
BoxParm =['alpha_gw_movmtn', 'effgw_rdg_resid','use_gw_rdg_resid']
BoxParm =['use_gw_convect_dp','use_gw_movmtn_pbl','use_gw_front','use_gw_rdg_resid','use_gw_rdg_beta','use_gw_oro']
BoxParm =['use_gw_movmtn_pbl','use_gw_rdg_resid','movmtn_source','alpha_gw_movmtn', 'effgw_movmtn_pbl','effgw_rdg_resid',]
BoxParm =['movmtn_source','alpha_gw_movmtn', 'effgw_movmtn_pbl','taubgnd','effgw_cm','effgw_rdg_beta','effgw_rdg_resid',]
#BoxParm =['use_gw_movmtn_pbl','movmtn_source','alpha_gw_movmtn', 'effgw_movmtn_pbl',]  #'effgw_rdg_resid',]


titles=[ r'1$^\circ$ control','14km simulation',r'1$^\circ$ w/ additional GW source','yy',]
titles=[ r'1$^\circ$ control',r'1$^\circ$ w/out GW parameterization','yy',]
ix=0

for A in Al:
    
    AA_sz = A['AA_sz'] 
    Lat_a , Zlev_a = np.meshgrid( A['latx'], A['zlev'] )
    AA_sz_comm = nuti.ZsY_x_ZtY( y_src=Zlev_a , x_src=Lat_a , data_src= AA_sz , y_tgt=CommonGrid['zlev'], x_tgt=CommonGrid['lat'] )

    AA_sz_comm = AA_sz_comm - Aval['AA_sz_comm']

    zlev_comm = CommonGrid['zlev']
    lat_comm  = CommonGrid['lat'] 
    mlev=dlev
    cmap='bwr' #_r'
    plotTop=65 #40 # 15 #10 #85
        
    years_a = A['years']
    if 'gw_parameters' in A:
        parameters=A['gw_parameters']
    else:
        parameters=None

    if ('desc' in A) and (use_desc_in_title == True):
        title = f"{A.desc} \n {season.upper()} {years_a[0]}-{years_a[-1]}"
    else:
        title = f"<{A.exp}> \n {season.upper()} {years_a[0]}-{years_a[-1]}"
    #title = titles[ix]

    n=ix+1 + n_next_startlabel
    Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=0.8 ) 
    ax1 = fig.add_axes( Axes1 )

    
    co1=ax1.contourf( lat_comm, zlev_comm, AA_sz_comm ,levels=mlev, cmap=cmap ,extend='both' )
    co2=ax1.contour(  lat_comm, zlev_comm, AA_sz_comm ,levels=mlev, colors='black')

    
    ax1.set_title( title ,fontsize=titlesize )
    ax1.set_ylim(0,plotTop)
    ax1.set_xlabel('Latitude', fontsize=16 )
    ax1.set_ylabel('Z (km)', fontsize=16 )
    cb=plt.colorbar(co1)
    ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
        fontsize=20, fontweight='bold', va='top')
    
    # Sponge layer hatching
    ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

    ax1.set_xlim( latlims )

    ix=ix+1

if (Aval['data_exists']==True):
    for ix in np.arange( start=numx,stop=numx+1 ):

        if (regrid_val_x_model == True) or (all_x_fv1x1_zERA==True):
            lon_v = Al[0]['lon_v']
            lat_v = Al[0]['lat_v']
            zlev_v = Al[0]['zlev_v']
            AAv_sz = Al[0]['AAv_sz']
            
        cmap='gist_ncar'
        title = f"<{valData}> {season.upper()} {years_Uv} " 
        n=ix+1+n_next_startlabel
        Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=0.8 ) 
        ax1 = fig.add_axes( Axes1 )
        co1=ax1.contourf(lat_v,zlev_v,AAv_sz ,levels=ulev, cmap=cmap , extend='both' )
        co2=ax1.contour(lat_v,zlev_v,AAv_sz ,levels=ulev, colors='black')
        ax1.set_title( title ,fontsize=titlesize )
        ax1.set_ylim(0,plotTop)
        ax1.set_xlabel('Latitude', fontsize=16 )
        ax1.set_ylabel('Z (km)', fontsize=16 )
        cb=plt.colorbar(co1)
        ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
            fontsize=20, fontweight='bold', va='top')
        ax1.set_xlim( latlims )

FigName = f"AGU_2024_Cecile_poster_zonalUT.png" #f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
FigName = f"multiFig.png" #f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
FigName = f"Figure_1.png" #f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
FigName = f"Fig_KeyResult4paper_{fld}_biases.png"

print( FigName) 
Pu.save_link(png_filename = FigName, dpi=600 )

In [ ]:
print(Al[2].X.lev/1000.)


In [ ]:
print( 100.*Al[0].X.lev.values[0:13])

print( Al[0]['zlev'][0:13])

In [ ]:
f= '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/f.e21.FWHISTBgcCrop.f09_f09_mg17.CMIP6-AMIP-WACCM.001/atm/proc/tseries/month_1/f.e21.FWHISTBgcCrop.f09_f09_mg17.CMIP6-AMIP-WACCM.001.cam.h0.U.195001-201412.nc'
poop=xr.open_dataset( f )


In [ ]:
poop.lev

In [ ]:
qrl0=Al[0].X.QRL.values
qrs0=Al[0].X.QRS.values
qrl1=Al[1].X.QRL.values
qrl2=Al[2].X.QRL.values
qrl3=Al[3].X.QRL.values
qrl4=Al[4].X.QRL.values

te3=Al[3].X.T.values
te2=Al[2].X.T.values
te1=Al[1].X.T.values
te0=Al[0].X.T.values


In [ ]:
plt.plot( 86_400.*qrs0[0,:,20000] )
plt.plot( 86_400.*qrl0[0,:,20000] )
plt.plot( 86_400.*qrl1[0,:,20000] )
plt.plot( 86_400.*qrl2[0,:,20000] )
plt.plot( 86_400.*qrl3[0,:,20000] )
plt.plot( 86_400.*qrl4[0,:,20000] )


In [ ]:
plt.plot( te1[:,0,20000] )
plt.plot( te2[:,0,20000] )
plt.plot( te3[:,0,20000] )


In [ ]:
print(dlev)

In [ ]:
print( Al[2].zlev)

In [ ]:

print( Al[2]['zlev'][-1] * 1000. )
print( Al[2]['zlev'][-1-5] * 1000. )
print( Al[2]['zlev'][-1-10] * 1000. )
print( Al[2]['zlev'][-1-20] * 1000. )
print( Al[2]['zlev'][-1-30] * 1000. )


In [ ]:

print( Al[2]['plev'][-1] * 100. )
print( Al[2]['plev'][-1-20] * 100. )
print( Al[2]['plev'][-1-30] * 100. )

print( Al[2]['plev'][-1-1] * 100. )
print( Al[2]['plev'][-1-1-10] * 100. )
print( Al[2]['plev'][-1-5] * 100. )
print( Al[2]['plev'][-1-5-10] * 100. )


In [ ]:
Al[0].X

In [ ]:
qrs=Al[2].X.QRS.values

In [ ]:
plt.plot( qrs[70,10,:] )

In [ ]:
print( Al[0].zlev[8] )
print( Al[0].zlev[12] )
print( Al[0].zlev[16] )

In [ ]:

plt.plot( Al[0].UU_sz[8,:] )
plt.plot( Al[1].UU_sz[8,:] )
plt.plot( Al[2].UU_sz[8,:] )
#plt.plot( Al[0].UU_sz[8,:] )
